In [87]:
import pandas as pd
import os


In [88]:
# df_raw = pd.read_csv('/home/swleocresearch/Desktop/triage-ai/datasets/extracted_letters_text.csv')
df_label = pd.read_csv('/home/swleocresearch/Desktop/triage-ai/datasets/labels.csv')

In [89]:
df_label.head()

,path,label
0,/home/swleocresearch/Desktop/triage-ai/dataset...,discharge
1,/home/swleocresearch/Desktop/triage-ai/dataset...,discharge
2,/home/swleocresearch/Desktop/triage-ai/dataset...,discharge
3,/home/swleocresearch/Desktop/triage-ai/dataset...,referral
4,/home/swleocresearch/Desktop/triage-ai/dataset...,surgery


In [119]:
import os
import pandas as pd
import re
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import torch 

def get_next_character(input_string, target_char):
    # Find the index of the target character
    index = input_string.find(target_char)
    
    # If the character is found and it's not the last character
    if index != -1 and index < len(input_string) - 1:
        # Return the next character
        return input_string[index + 1]
    else:
        return None  # Return None if the character is not found or it's the last character


def extract_text_from_pdf(pdf_path):
    # print(f"Extracting text from: {pdf_path}")
    doc = DocumentFile.from_pdf(pdf_path)
    device = torch.device("cuda:0")
    predictor = ocr_predictor(pretrained=True).to(device)
    result = predictor(doc)
    json_output = result.export()
    
    extracted_text = ""
    for page in json_output['pages']:
        for block in page['blocks']:
            for line in block['lines']:
                for word in line['words']:
                    extracted_text += word['value'] + ' '
                extracted_text += '\n'
    
    return extracted_text


In [121]:
import os
import pandas as pd
from tqdm import tqdm

rootdir = '/home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/'
df = pd.DataFrame(columns=['referral', 'output'])

for subdir, dirs, files in tqdm(os.walk(rootdir)):
    for file in files:
        file_path = os.path.join(subdir, file)
        file_id = file.split('/')[0].split(' ')[-1].split('.')[0]
        # print('file_id: ', file_id)
        
        if 'R' in file_id:
            referral_text = extract_text_from_pdf(file_path)
            next_char = get_next_character(file_id, 'R')
            ref_id = f'{next_char}'
            
            # print('subdir: ', subdir)
            for subdir1, dirs1, files1 in os.walk(subdir):
                for f in files1:
                    file_id = f.split('/')[0].split(' ')[-1].split('.')[0]
                    
                    if f'C{ref_id}' in f:
                        clinic_file_path = os.path.join(subdir1, f)
                        label_row = df_label.loc[df_label['path'] == clinic_file_path, 'label']
                        
                        if not label_row.empty:
                            label = label_row.iat[0]
                            print(f"Label found: {label}")
                        else:
                            print(f"No label found for {clinic_file_path}")
                            continue  # Skip this iteration if no label is found

            
                        print(f'input_file: {file_path}, label_file: {clinic_file_path}')
                        
                        new_row = pd.DataFrame({'referral': [referral_text], 'output': [label]})
                        df = pd.concat([df, new_row], ignore_index=True)

# At the end, if you'd like to see the dataframe
df.to_csv('triage_dataset.csv')

2it [00:05,  2.84s/it]

Label found: surgery
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40364725/2023-03-07 40364725 R1.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40364725/2023-05-17 40364725 C1.pdf


3it [00:07,  2.36s/it]

Label found: referral
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40199374/2021-01-25 40199374 R1.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40199374/2021-04-30 40199374 C1.pdf
Label found: surgery
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40010910/2023-02-13 40010910 R1.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40010910/2023-05-18 40010910 C1.pdf


4it [00:13,  3.69s/it]

...

Label found: tests
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40083586/2023-02-02 40083586 R1.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40083586/2023-04-19 40083586 C1.pdf


62it [06:14,  6.16s/it]

Label found: surgery
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40329532/2023-02-07 40329532 R1.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40329532/2023-04-03 40329532 C1.pdf
Label found: tests
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40215441/2023-03-23 40215441 R2.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40215441/2023-07-10 40215441 C2.pdf
Label found: discharge
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40215441/2023-05-15 40215441 R3.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40215441/2023-10-13 40215441 C3.pdf


63it [06:24,  6.10s/it]

Label found: tests
input_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40215441/2023-03-16 40215441 R1.pdf, label_file: /home/swleocresearch/Desktop/triage-ai/datasets/labelled_dataset/40215441/2023-06-16 40215441 C1.pdf


In [130]:
df_written = pd.read_csv('/home/swleocresearch/Desktop/triage-ai/datasets/triage_dataset.csv')

In [131]:
df_written['output'].value_counts()

output
surgery      33
discharge    29
tests        18
fu           12
referral      6
injection     6
physio        4
Name: count, dtype: int64

In [134]:
df_written.iloc[:,2].tolist()

['surgery',
 'referral',
 'surgery',
 'physio',
 'surgery',
 'discharge',
 'tests',
 'discharge',
 'surgery',
 'discharge',
 'referral',
 'discharge',
 'discharge',
 'surgery',
 'discharge',
 'surgery',
 'tests',
 'surgery',
 'injection',
 'fu',
 'tests',
 'discharge',
 'discharge',
 'physio',
 'surgery',
 'fu',
 'referral',
 'injection',
 'surgery',
 'surgery',
 'tests',
 'injection',
 'discharge',
 'surgery',
 'surgery',
 'fu',
 'referral',
 'discharge',
 'discharge',
 'fu',
 'discharge',
 'tests',
 'surgery',
 'surgery',
 'discharge',
 'referral',
 'discharge',
 'surgery',
 'surgery',
 'tests',
 'discharge',
 'discharge',
 'surgery',
 'surgery',
 'discharge',
 'surgery',
 'tests',
 'surgery',
 'fu',
 'surgery',
 'surgery',
 'fu',
 'surgery',
 'surgery',
 'surgery',
 'physio',
 'discharge',
 'discharge',
 'injection',
 'fu',
 'discharge',
 'tests',
 'injection',
 'referral',
 'discharge',
 'tests',
 'physio',
 'discharge',
 'surgery',
 'tests',
 'discharge',
 'fu',
 'surgery',
 'fu',

In [ ]:

# Example array to iterate over

# Create an empty DataFrame with two columns

# Iterate through the array and add values to the DataFrame
for i in range(len(array)):
    df = df.append({'Column1': array[i], 'Column2': array[i] * 2}, ignore_index=True)

# Display the resulting DataFrame
print(df)
